In [1]:
import numpy as np
from scipy.optimize import linprog
from itertools import *

def powerset(iterable):
    "powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [2]:
constraints = 38
A = np.empty((constraints, 22)) # a1, a2, a3, a4, a5, a6, ar; b; c; nu
b = np.empty((constraints),)
c = np.zeros((22,))

In [3]:
delta = 0.001
epsilon = 0.00001

In [4]:
c[21] = -1

In [5]:
# eqns 4.2
A[0] = [1, 2, 3, 4, 5, 6, 7,  0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0,  0] 
A[1] = [0, 0, 0, 0, 0, 0, 0,  1, 2, 3, 4, 5, 6, 7,  0, 0, 0, 0, 0, 0, 0,  0] 
A[2] = [0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0,  1, 2, 3, 4, 5, 6, 7,  0]

b[0] = b[1] = b[2] = 1

In [6]:
# eqns 4.3
A[3] = [-1, -1, -1, -1, -1, -1, -1,  -1, -1, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  0]
A[4] = [-1, -1, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, -1, -1, -1,  0]
A[5] = [0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, -1, -1, -1,  -1, -1, -1, -1, -1, -1, -1,  0]

b[3] = b[4] = b[5] = epsilon ** 2 - 0.66

In [7]:
# eqn 4.4
A[6] = [1, 1, 1, 1, 1, 1, 1,  1, 1, 1, 1, 1, 1, 1,  1, 1, 1, 1, 1, 1, 1,  0]
b[6] = 1 + delta - epsilon

In [8]:
# eqn 4.5
A[7] = [1, 1, 1, 1, 1, 1, 1,  0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0,  0]
A[8] = [0, 0, 0, 0, 0, 0, 0,  1, 1, 1, 1, 1, 1, 1,  0, 0, 0, 0, 0, 0, 0,  0]
A[9] = [0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0,  1, 1, 1, 1, 1, 1, 1,  0]
b[7] = b[8] = b[9] = 0.34 + delta - epsilon / 2

In [9]:
# thue
A[10] = [0, 1, 0, 1, 0, 1, 0,  0, 1, 0, 1, 0, 1, 0,  0, 0, 0, 0, 0, 0, 0,  1]
A[11] = [0, 0, 1, 0, 0, 1, 0,  0, 0, 1, 0, 0, 1, 0,  0, 0, 0, 0, 0, 0, 0,  1]
A[12] = [0, 0, 0, 0, 1, 0, 0,  0, 0, 0, 0, 1, 0, 0,  0, 0, 0, 0, 0, 0, 0,  1]

A[13] = [0, 1, 0, 1, 0, 1, 0,  0, 0, 0, 0, 0, 0, 0,  0, 1, 0, 1, 0, 1, 0,  1]
A[14] = [0, 0, 1, 0, 0, 1, 0,  0, 0, 0, 0, 0, 0, 0,  0, 0, 1, 0, 0, 1, 0,  1]
A[15] = [0, 0, 0, 0, 1, 0, 0,  0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 1, 0, 0,  1]

A[16] = [0, 0, 0, 0, 0, 0, 0,  0, 1, 0, 1, 0, 1, 0,  0, 1, 0, 1, 0, 1, 0,  1]
A[17] = [0, 0, 0, 0, 0, 0, 0,  0, 0, 1, 0, 0, 1, 0,  0, 0, 1, 0, 0, 1, 0,  1]
A[18] = [0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 1, 0, 0,  0, 0, 0, 0, 1, 0, 0,  1]

b[10] = b[11] = b[12] = b[13] = b[14] = b[15] = b[16] = b[17] = b[18] = 1 + delta

In [10]:
# weak fourier
A[19] = [-1, -1, -1, -1, -1, 0, -1,  -1, -1, -1, -1, -1, 0, -1,  0, 0, 0, 0, 0, 0, 0,  1]
A[20] = [-1, -1, -1, -1, 0, -1, -1,  -1, -1, -1, -1, 0, -1, -1,  0, 0, 0, 0, 0, 0, 0,  1]
A[21] = [-1, -1, -1, 0, -1, -1, -1,  -1, -1, -1, 0, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  1]
A[22] = [-1, -1, 0, -1, -1, -1, -1,  -1, -1, 0, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  1]
A[23] = [-1, 0, -1, -1, -1, -1, -1,  -1, 0, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  1]
A[24] = [0, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  1]

A[25] = [-1, -1, -1, -1, -1, 0, -1,  0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, -1, 0, -1,  1]
A[26] = [-1, -1, -1, -1, 0, -1, -1,  0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, 0, -1, -1,  1]
A[27] = [-1, -1, -1, 0, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, 0, -1, -1, -1,  1]
A[28] = [-1, -1, 0, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  -1, -1, 0, -1, -1, -1, -1,  1]
A[29] = [-1, 0, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  -1, 0, -1, -1, -1, -1, -1,  1]
A[30] = [0, -1, -1, -1, -1, -1, -1,  0, 0, 0, 0, 0, 0, 0,  0, -1, -1, -1, -1, -1, -1,  1]

A[31] = [0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, -1, 0, -1,  -1, -1, -1, -1, -1, 0, -1,  1]
A[32] = [0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, -1, 0, -1, -1,  -1, -1, -1, -1, 0, -1, -1,  1]
A[33] = [0, 0, 0, 0, 0, 0, 0,  -1, -1, -1, 0, -1, -1, -1,  -1, -1, -1, 0, -1, -1, -1,  1]
A[34] = [0, 0, 0, 0, 0, 0, 0,  -1, -1, 0, -1, -1, -1, -1,  -1, -1, 0, -1, -1, -1, -1,  1]
A[35] = [0, 0, 0, 0, 0, 0, 0,  -1, 0, -1, -1, -1, -1, -1,  -1, 0, -1, -1, -1, -1, -1,  1]
A[36] = [0, 0, 0, 0, 0, 0, 0,  0, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1,  1]

b[19:37] = (1 + delta) / 2

In [11]:
A[37] = [1, 1, 0, 0, 0, 0, 0,  1, 1, 0, 0, 0, 0, 0,  1, 1, 0, 0, 0, 0, 0,  0]
b[37] = 0.34 + delta

In [12]:
# determinant
shifts = [(0, 7), (0, 14), (7, 14)]
for x, y in shifts:
    for p in range(1, 7):
        for q in range(1, 7):
            row1 = np.zeros((22,))
            row2 = np.zeros((22,))
            row1[21] = row2[21] = 1
            row1[p - 1 + x] = 1 - 1 / q
            row1[q - 1 + y] = 1

            row2[p - 1 + x] = 1
            row2[q - 1 + y] = 1 - 1 / p
            A = np.vstack([A, row1, row2])
            b = np.hstack([b, 1 + delta, 1 + delta])
print(A.shape)

(254, 22)


In [13]:
geom = np.empty((2 ** 21, 22))
geom_b = np.empty((2 ** 21,))
geom_b[:] = 1 + delta

count = 0

# geometry
shifts = [(0, 7), (0, 14), (7, 14)]
for I1, I2, I3 in product(powerset(range(1, 8)), repeat=3):
    # if len(I1) > 5 and len(I2) > 5 and len(I3) > 5:
    row = np.zeros((22,))
    row[21] = 1
    for i in I1:
        row[i - 1] = 1 - 2 / 3 * i
    for i in I2:
        row[i - 1 + 7] = 1 - 2 / 3 * i
    for i in I3:
        row[i - 1 + 14] = 1 - 2 / 3 * i
    geom[count] = row
    count += 1

A = np.vstack([A, geom])
b = np.hstack([b, geom_b])

print(A.shape)

(2097406, 22)


In [14]:
count

2097152

In [15]:
sol = linprog(c, A, b)

In [16]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -0.8511739800411859
              x: [ 1.078e-01  5.907e-03 ...  8.796e-03  8.512e-01]
            nit: 57
          lower:  residual: [ 1.078e-01  5.907e-03 ...  8.796e-03
                              8.512e-01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  1.196e+00
                              1.160e+00]
                 marginals: [-6.266e-02 -6.266e-02 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mi

In [17]:
print(f"ν = {sol.x[21]}")

ν = 0.8511739800411859


In [18]:
print("a =", sol.x[0:7])

a = [0.10775982 0.00590685 0.07491301 0.06900616 0.06361817 0.
 0.008796  ]


In [19]:
print("b =", sol.x[7:14])

b = [0.10775982 0.00590685 0.07491301 0.06900616 0.06361817 0.
 0.008796  ]


In [20]:
print("c =", sol.x[14:21])

c = [0.10775982 0.00590685 0.07491301 0.06900616 0.06361817 0.
 0.008796  ]


In [21]:
np.where(sol.slack == 0)[0]

array([  0,   1,   2,   3,   4,   5,  10,  11,  13,  14,  16,  17,  37,
        44,  46,  75,  87, 116, 118, 147, 159, 188, 190, 219, 231])